In [1]:
import pandas as pd

In [2]:
company_list = [
    "聯電",
    "日月光",
    "聯發科",
    "台積電",
    "台達電"
]

In [3]:
def get_data(source, company):
    source_dict = {
        'news':'新聞已預測.xlsx',
        'posts':'社群貼文已預測.xlsx',
    }
    source_df = pd.read_excel(rf'sent_score/origin_data/{source_dict[source]}',sheet_name=company)
    df = source_df.copy()
    if source == 'news':
        df = df[['日期', '預測情緒標籤']]
    else:
        df = df[['時間', '預測情緒標籤']]
        df.rename(columns={'時間':'日期'}, inplace=True)
    df['預測情緒標籤'] = df['預測情緒標籤']-1
    return df
    
    

In [4]:
def sentimental_score(df, source):
    labels = []

    sum_label = df.groupby('日期').apply(lambda x: x.sum())
    labels.append(sum_label['預測情緒標籤'].rename(f'{source}_sc_sum'))

    mean_label = df.groupby('日期').apply(lambda x: round(x.mean(), 4))
    labels.append(mean_label['預測情緒標籤'].rename(f'{source}_sc_mean'))

    count_label = df.groupby('日期').apply(lambda x: x.count())
    labels.append(count_label['預測情緒標籤'].rename(f'{source}_sc_count'))

    labels_df = pd.concat(labels, axis=1)
    
    return labels_df['2022-01-01':'2022-12-31']

In [5]:
writer = pd.ExcelWriter(r'sent_score\sentimental_score.xlsx')
for company in company_list:
    news_df = get_data('news', company)
    posts_df = get_data('posts', company)
    social_df = pd.concat([news_df, posts_df]) 

    posts_score = sentimental_score(posts_df, 'post')
    news_score = sentimental_score(news_df, 'news')
    social_score = sentimental_score(social_df, 'social')

    df = pd.concat([posts_score, news_score, social_score], axis=1).fillna(0)
    df.to_excel(writer, index=True, sheet_name=company)
    
writer._save()
